<a href="https://colab.research.google.com/github/SiweiLuo/DL_with_Python/blob/master/up_and_running_with_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np 
import os 

def reset_graph(seed=42):
  tf.reset_default_graph()
  tf.set_random_seed(seed)
  np.random.seed(seed)
  
%matplotlib inline 
  
import matplotlib
import matplotlib.pyplot as plt 
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

def save_fig(fig_id,tight_layout=True):
  path = os.path.join(fig_id+".png")
  print("Saving figure", fig_id)
  if tight_layout:
      plt.tight_layout()
  plt.savefig(path, format='png', dpi=300)


In [3]:
import tensorflow as tf 

reset_graph() 

x = tf.Variable(3,name="x")
y = tf.Variable(4,name="y")

f = x*x*y + y + 2 

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result) 


42


In [0]:
sess.close()

In [0]:
with tf.Session() as sess:
  x.initializer.run()
  y.initializer.run()
  results = f.eval()

In [7]:
result

42

In [0]:
init = tf.global_variables_initializer() 

with tf.Session() as sess: 
  init.run()
  result = f.eval() 

In [9]:
result

42

In [0]:
init = tf.global_variables_initializer()


In [11]:
sess = tf.InteractiveSession()
init.run() 
result = f.eval()
print(result)

42


In [0]:
sess.close()


In [13]:
result

42

In [15]:
reset_graph()
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [16]:
graph = tf.Graph()
with graph.as_default():
  x2 = tf.Variable(2)
  
x2.graph is graph 

True

In [17]:
x2.graph is tf.get_default_graph()

False

In [18]:
w = tf.constant(3)
x = w+2
y = x+5
z = x*3 

with tf.Session() as sess:
  print(y.eval()) 
  print(z.eval())

10
15


In [19]:
with tf.Session() as sess:
  y_val, z_val = sess.run([y,z]) 
  print(y_val)
  print(z_val)

10
15


In [20]:
import numpy as np 
from sklearn.datasets import fetch_california_housing 

reset_graph()

housing = fetch_california_housing()
m,n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)),housing.data]

x = tf.constant(housing_data_plus_bias,dtype=tf.float32,name="x")
y = tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")
xt = tf.transpose(x)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(xt,x)),xt),y)

with tf.Session() as sess:
  theta_value = theta.eval()

In [21]:
theta_value

array([[-3.71037292e+01],
       [ 4.36282694e-01],
       [ 9.40542948e-03],
       [-1.06901854e-01],
       [ 6.43611908e-01],
       [-4.06625077e-06],
       [-3.78273334e-03],
       [-4.23094332e-01],
       [-4.36462164e-01]], dtype=float32)

In [23]:
x = housing_data_plus_bias
y = housing.target.reshape(-1,1)
theta_numpy = np.linalg.inv(x.T.dot(x)).dot(x.T).dot(y)

print(theta_numpy)


[[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]


In [24]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing.data,housing.target.reshape(-1,1))

print(np.r_[lin_reg.intercept_.reshape(-1,1),lin_reg.coef_.T])

[[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]


In [0]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m,1)),scaled_housing_data]

In [28]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01 

x = tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32,name="x")
y = tf.constant(housing.target.reshape(-1, 1),dtype=tf.float32,name="y")
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0,seed=42),name="theta")

y_pred = tf.matmul(x,theta,name="predictions")
error = y_pred-y
mse = tf.reduce_mean(tf.square(error),name="mse")
gradients = 2/m * tf.matmul(tf.transpose(x),error)
training_op = tf.assign(theta,theta-learning_rate*gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess: 
  sess.run(init)
  
  for epoch in range(n_epochs):
    if epoch % 100 ==0: 
      print("Epoch",epoch, "MSE = ",mse.eval())
      
    sess.run(training_op)
    
  best_theta = theta.eval()

Epoch 0 MSE =  9.161542
Epoch 100 MSE =  0.71450055
Epoch 200 MSE =  0.56670487
Epoch 300 MSE =  0.55557173
Epoch 400 MSE =  0.5488112
Epoch 500 MSE =  0.5436363
Epoch 600 MSE =  0.53962904
Epoch 700 MSE =  0.5365092
Epoch 800 MSE =  0.5340677
Epoch 900 MSE =  0.5321473


In [29]:
best_theta

array([[ 2.0685523 ],
       [ 0.8874027 ],
       [ 0.14401656],
       [-0.3477088 ],
       [ 0.36178362],
       [ 0.00393811],
       [-0.04269556],
       [-0.66145283],
       [-0.6375278 ]], dtype=float32)

In [0]:
reset_graph()

n_epochs = 1000 

learning_rate = 0.01 


x = tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32,name="x")
y = tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0,seed=42),name="theta")
y_pred = tf.matmul(x,theta,name="predictions")
error = y_pred - y 
mse = tf.reduce_mean(tf.square(error),name="mse")

In [0]:
gradients = tf.gradients(mse,[theta])[0]

In [32]:
training_op = tf.assign(theta,theta-learning_rate*gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess: 
  sess.run(init)
  
  for epoch in range(n_epochs):
    if epoch % 100 == 0: 
      print("Epoch",epoch,"MSE = ",mse.eval())
      
    sess.run(training_op)
    
print("Best theta:",best_theta)

Epoch 0 MSE =  9.161542
Epoch 100 MSE =  0.7145004
Epoch 200 MSE =  0.56670487
Epoch 300 MSE =  0.55557173
Epoch 400 MSE =  0.5488112
Epoch 500 MSE =  0.5436363
Epoch 600 MSE =  0.53962904
Epoch 700 MSE =  0.5365092
Epoch 800 MSE =  0.5340677
Epoch 900 MSE =  0.5321473
Best theta: [[ 2.0685523 ]
 [ 0.8874027 ]
 [ 0.14401656]
 [-0.3477088 ]
 [ 0.36178362]
 [ 0.00393811]
 [-0.04269556]
 [-0.66145283]
 [-0.6375278 ]]


In [0]:
def my_func(a,b):
  z = 0 
  
  for i in range(100):
    z = a*np.cos(z+i)+z*np.sin(b-i)
    
  return z 


In [34]:
my_func(0.2,0.3)

-0.21253923284754914

In [0]:
reset_graph()

a = tf.Variable(0.2,name="a")
b = tf.Variable(0.3,name="b")
z = tf.Variable(0.0,name="z0")

for i in range(100):
  z = a * tf.cos(z+i) + z * tf.sin(b - i)
  
grads = tf.gradients(z,[a,b])

init = tf.global_variables_initializer()

In [36]:
with tf.Session() as sess:
  init.run()
  print(z.eval())
  print(sess.run(grads))

-0.21253741
[-1.1388494, 0.19671395]


In [0]:
reset_graph()

n_epochs = 1000 
learning_rate = 0.01 

x = tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32,name="x")
y = tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0,seed=42),name="theta")
y_pred = tf.matmul(x,theta,name="predictions")
error = y_pred-y
mse = tf.reduce_mean(tf.square(error),name="mse")



In [0]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

In [39]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
  sess.run(init)
  
  for epoch in range(n_epochs):
    if epoch % 100 == 0:
      print("Epoch",epoch,"MSE =",mse.eval())
    sess.run(training_op)
    
  best_theta = theta.eval()
  
print(best_theta)

Epoch 0 MSE = 9.161542
Epoch 100 MSE = 0.71450055
Epoch 200 MSE = 0.56670487
Epoch 300 MSE = 0.55557173
Epoch 400 MSE = 0.5488112
Epoch 500 MSE = 0.5436363
Epoch 600 MSE = 0.53962904
Epoch 700 MSE = 0.5365092
Epoch 800 MSE = 0.5340677
Epoch 900 MSE = 0.5321473
[[ 2.0685523 ]
 [ 0.88740265]
 [ 0.14401655]
 [-0.34770873]
 [ 0.3617836 ]
 [ 0.00393811]
 [-0.04269556]
 [-0.6614529 ]
 [-0.6375279 ]]


In [0]:
reset_graph()

n_epochs = 1000

learning_rate = 0.01 


x = tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32,name="x")
y = tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0,seed=42),name="theta")

y_pred = tf.matmul(x,theta,name="predictions")
error = y_pred - y 
mse = tf.reduce_mean(tf.square(error),name="mse")

In [0]:
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=0.9)

In [0]:
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [0]:
with tf.Session() as sess:
  sess.run(init)
  
  for epoch in range(n_epochs):
    sess.run(training_op)
    
  best_theta = theta.eval()
  
  

In [45]:
reset_graph()

A = tf.placeholder(tf.float32,shape=(None,3))
B = A + 5 

with tf.Session() as sess:
  B_val_1 = B.eval(feed_dict={A:[[1,2,3]]})
  B_val_2 = B.eval(feed_dict={A:[[4,5,6],[7,8,9]]})
  
print(B_val_1)

[[6. 7. 8.]]


In [46]:
print(B_val_2)

[[ 9. 10. 11.]
 [12. 13. 14.]]


In [0]:
n_epochs = 1000
learning_rate = 0.01 


In [0]:
reset_graph()

x = tf.placeholder(tf.float32,shape=(None,n+1),name="x")
y = tf.placeholder(tf.float32,shape=(None,1),name="y")

In [0]:
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0,seed=42),name="theta")
y_pred = tf.matmul(x,theta,name="predictions")
error = y_pred - y 
mse = tf.reduce_mean(tf.square(error),name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [0]:
n_epochs=10
batch_size = 100
n_batches = int(np.ceil(m/batch_size))

In [0]:
def fetch_batch(epoch,batch_index,batch_size):
  np.random.seed(epoch*n_batches+batch_index)
  indices = np.random.randint(m,size=batch_size)
  x_batch = scaled_housing_data_plus_bias[indices]
  y_batch = housing.target.reshape(-1,1)[indices]
  return x_batch,y_batch

with tf.Session() as sess:
  sess.run(init)
  for epoch in range(n_epochs):
    for batch_index in range(n_batches):
      x_batch,y_batch = fetch_batch(epoch,batch_index,batch_size)
      sess.run(training_op,feed_dict={x:x_batch,y:y_batch})
  best_theta = theta.eval()

In [0]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01 

x = tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32,name="x")
y = tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0,seed=42),name="theta")
y_pred = tf.matmul(x,theta,name="predictions")

error = y_pred - y 

mse = tf.reduce_mean(tf.square(error),name="mse")

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)

training_op = optimizer.minimize(mse) 

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
  sess.run(init)
  for epoch in range(n_epochs):
    if epoch%100==0:
      save_path = saver.save(sess,"my_model.ckpt")
    sess.run(training_op)
    
  best_theta = theta.eval()
  save_path = saver.save(sess,"my_model_final.ckpt")
  
  
      

In [53]:
ls

checkpoint                         my_model_final.ckpt.data-00000-of-00001
my_model.ckpt.data-00000-of-00001  my_model_final.ckpt.index
my_model.ckpt.index                my_model_final.ckpt.meta
my_model.ckpt.meta                 sample_data/


In [54]:
with tf.Session() as sess:
  saver.restore(sess,"my_model_final.ckpt")
  best_theta_restored = theta.eval()
  

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from my_model_final.ckpt


In [55]:
np.allclose(best_theta,best_theta_restored)

True

In [0]:
saver = tf.train.Saver({"weights":theta})

In [58]:
reset_graph()

saver = tf.train.import_meta_graph("my_model_final.ckpt.meta")

theta = tf.get_default_graph().get_tensor_by_name("theta:0")

with tf.Session() as sess:
  saver.restore(sess,"my_model_final.ckpt")
  best_theta_restored = theta.eval()

INFO:tensorflow:Restoring parameters from my_model_final.ckpt


In [0]:
reset_graph()

from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir,now)

In [0]:

n_epochs = 1000
learning_rate = 0.01 

x = tf.placeholder(tf.float32,shape=(None,n+1),name="x")
y = tf.placeholder(tf.float32,shape=(None,1),name="y")
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0,seed=42),name="theta")
y_pred = tf.matmul(x,theta,name="predictions")
error = y_pred - y 
mse = tf.reduce_mean(tf.square(error),name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [0]:
mse_summary = tf.summary.scalar('MSE',mse)
file_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())

In [0]:
n_epochs = 10 
batch_size = 100 
n_batches = int(np.ceil(m/batch_size))

In [0]:
with tf.Session() as sess:
  sess.run(init)
  
  for epoch in range(n_epochs):
    for batch_index in range(n_batches):
      x_batch,y_batch = fetch_batch(epoch,batch_index,batch_size)
      if batch_index % 10 == 0: 
        summary_str = mse_summary.eval(feed_dict={x:x_batch,y:y_batch})
        step = epoch*n_batches + batch_index 
        file_writer.add_summary(summary_str,step)
      sess.run(training_op,feed_dict={x:x_batch,y:y_batch})
      
  best_theta = theta.eval()

In [0]:
reset_graph() 

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir,now)

n_epochs = 1000
learning_rate = 0.01 

x = tf.placeholder(tf.float32,shape=(None,n+1),name="x")
y = tf.placeholder(tf.float32,shape=(None,1),name="y")
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0,seed=42),name="theta")
y_pred = tf.matmul(x,theta,name="predictions")



In [0]:
with tf.name_scope("loss") as scope:
  error = y_pred - y 
  mse = tf.reduce_mean(tf.square(error),name="mse")

In [0]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

mse_summary = tf.summary.scalar('MSE',mse)
file_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())

In [70]:
n_epochs = 10 
batch_size = 100
n_batches = int(np.ceil(m/batch_size))

with tf.Session() as sess:
  sess.run(init)
  
  for epoch in range(n_epochs):
    for batch_index in range(n_batches):
      x_batch,y_batch = fetch_batch(epoch,batch_index,batch_size)
      if batch_index % 10 == 0 : 
        summary_str = mse_summary.eval(feed_dict={x:x_batch,y:y_batch})
        step = epoch * n_batches + batch_index 
        file_writer.add_summary(summary_str,step)
      sess.run(training_op,feed_dict={x:x_batch,y:y_batch})
      
  best_theta = theta.eval()
  
file_writer.flush()
file_writer.close()

print(best_theta)

[[ 2.0703337 ]
 [ 0.8637145 ]
 [ 0.12255149]
 [-0.31211886]
 [ 0.38510385]
 [ 0.00434167]
 [-0.01232954]
 [-0.83376896]
 [-0.8030471 ]]


In [71]:
reset_graph()

a1 = tf.Variable(0,name="a")
a2 = tf.Variable(0,name="a")

with tf.name_scope("param"):
  a3 = tf.Variable(0,name="a")
  
with tf.name_scope("param"):
  a4 = tf.Variable(0,name="a")
  
for node in (a1,a2,a3,a4):
  print(node.op.name)

a
a_1
param/a
param_1/a


In [0]:
reset_graph()

n_features = 3 

x = tf.placeholder(tf.float32,shape=(None,n_features),name="x")

w1 = tf.Variable(tf.random_normal((n_features,1)),name="weights1")
w2 = tf.Variable(tf.random_normal((n_features,1)),name="weights2")
b1 = tf.Variable(0.0,name="bias1")
b2 = tf.Variable(0.0,name="bias2")

z1 = tf.add(tf.matmul(x,w1),b1,name="z1")
z2 = tf.add(tf.matmul(x,w2),b2,name="z2")

relu1 = tf.maximum(z1,0.,name="relu1")
relu2 = tf.maximum(z1,0.,name="relu2")

output = tf.add(relu1,relu2,name="output")

In [0]:
reset_graph() 

def relu(x):
  w_shape = (int(x.get_shape()[1]),1)
  w = tf.Variable(tf.random_normal(w_shape),name="weights")
  b = tf.Variable(0.0,name="bias")
  z = tf.add(tf.matmul(x,w),b,name="z")
  return tf.maximum(z,0.,name="relu")

n_features = 3 

x = tf.placeholder(tf.float32,shape=(None,n_features),name="x")
relus = [relu(x) for i in range(5)]
output = tf.add_n(relus,name="output")

In [0]:
file_writer = tf.summary.FileWriter("logs/relu1",tf.get_default_graph())

In [0]:
reset_graph()

def relu(x):
  with tf.name_scope("relu"):
    w_shape = (int(x.get_shape()[1]),1)
    w = tf.Variable(tf.random_normal(w_shape),name="weights")
    b = tf.Variable(0.0,name="bias")
    z = tf.add(tf.matmul(x,w),b,name="z")
    return tf.maximum(z,0.,name="max")

In [0]:
n_features = 3 

x = tf.placeholder(tf.float32,shape=(None,n_features),name="x")
relus = [relu(x) for i in range(5)]
output = tf.add_n(relus,name="output")

file_writer = tf.summary.FileWriter("logs/relu2",tf.get_default_graph())
file_writer.close()

In [0]:
reset_graph()

def relu(x,threshold):
  with tf.name_scope("relu"):
    w_shape = (int(x.get_shape()[1]),1)
    w = tf.Variable(tf.random_normal(w_shape),name="weights")
    b = tf.Variable(0.0,name="bias")
    z = tf.add(tf.matmul(x,w),b,name="z")
    return tf.maximum(z,threshold,name="max")
  
threshold = tf.Variable(0.0,name="threshold")
x = tf.placeholder(tf.float32,shape=(None,n_features),name="x")
relus = [relu(x,threshold) for i in range(5)]
output = tf.add_n(relus,name="output")

In [0]:
reset_graph()

def relu(x):
  with tf.name_scope("relu"):
    if not hasattr(relu,"threshold"):
      relu.threshold = tf.Variable(0.0,name="threshold")
    w_shape = int(x.get_shape()[1]),1
    w = tf.Variable(tf.random_normal(w_shape),name="weights")
    b = tf.Variable(0.0,name="bias")
    z = tf.add(tf.matmul(x,w),b,name="z")
    return tf.maximum(z,relu.threshold,name="max")

In [0]:
x = tf.placeholder(tf.float32,shape=(None,n_features),name="x")
relus = [relu(x) for i in range(5)]
output = tf.add_n(relus,name="output")


In [0]:
reset_graph()

with tf.variable_scope("relu"):
  threshold = tf.get_variable("threshold",shape=(),
                             initializer = tf.constant_initializer(0.0))
  

In [0]:
with tf.variable_scope("relu",reuse=True):
  threshold = tf.get_variable("threshold")

In [0]:
with tf.variable_scope("relu") as scope:
  scope.reuse_variables()
  threshold = tf.get_variable("threshold")
  

In [0]:
reset_graph()

def relu(x):
  with tf.variable_scope("relu",reuse=True):
    threshold = tf.get_variable("threshold")
    w_shape = int(x.get_shape()[1]),1
    w = tf.Variable(tf.random_normal(w_shape),name="weights")
    b = tf.Variable(0.0,name="bias")
    z = tf.add(tf.matmul(x,w),b,name="z")
    return tf.maximum(z,threshold,name="max")
  
x = tf.placeholder(tf.float32,shape=(None,n_features),name="x")
with tf.variable_scope("relu"):
  threshold = tf.get_variable("threshold",shape=(),
                             initializer=tf.constant_initializer(0.0))
  
relus = [relu(x) for relu_index in range(5)]
output = tf.add_n(relus,name="output")

In [0]:
file_writer = tf.summary.FileWriter("logs/relu6",tf.get_default_graph())
file_writer.close()

In [0]:
reset_graph()

def relu(x):
  with tf.variable_scope("relu"):
    threshold = tf.get_variable("threshold",shape=(),initializer=tf.constant_initializer(0.0))
    w_shape = (int(x.get_shape()[1]),1)
    w = tf.Variable(tf.random_normal(w_shape),name="weights")
    b = tf.Variable(0.0,name="bias")
    z = tf.add(tf.matmul(x,w),b,name="z")
    return tf.maximum(z,threshold,name="max")
  
x = tf.placeholder(tf.float32,shape=(None,n_features),name="x")

with tf.variable_scope("",default_name="") as scope:
  first_relu = relu(x)
  scope.reuse_variables()
  relus = [first_relu] + [relu(x) for i in range(4)]
  
output = tf.add_n(relus,name="output")

file_writer = tf.summary.FileWriter("logs/relu8",tf.get_default_graph())
file_writer.close()

In [0]:
reset_graph()

def relu(x):
  threshold = tf.get_variable("threshold",shape=(),
                             initializer=tf.constant_initializer(0.0))
  
  w_shape = (int(x.get_shape()[1]),1)
  w = tf.Variable(tf.random_normal(w_shape),name="weights")
  b = tf.Variable(0.0,name="bias")
  z = tf.add(tf.matmul(x,w),b,name="z")
  return tf.maximum(z,threshold,name="max")

x = tf.placeholder(tf.float32,shape=(None,n_features),name="x")
relus = []
for relu_index in range(5):
  with tf.variable_scope("relu",reuse=(relu_index>=1)) as scope:
    relus.append(relu(x))
output = tf.add_n(relus,name="output")
  

In [0]:
file_writer = tf.summary.FileWriter("logs/relu9",tf.get_default_graph())
file_writer.close()


In [91]:
reset_graph()

with tf.variable_scope("my_scope"):
  x0 = tf.get_variable("x",shape=(),initializer=tf.constant_initializer(0.))
  x1 = tf.Variable(0.,name="x")
  x2 = tf.Variable(0.,name="x")
  
with tf.variable_scope("my_scope",reuse=True):
  x3 = tf.get_variable("x")
  x4 = tf.Variable(0.,name="x")
  
with tf.variable_scope("",default_name="",reuse=True):
  x5 = tf.get_variable("my_scope/x")
  
  
print("x0:", x0.op.name)
print("x1:", x1.op.name)
print("x2:", x2.op.name)
print("x3:", x3.op.name)
print("x4:", x4.op.name)
print("x5:", x5.op.name)
print(x0 is x3 and x3 is x5)

x0: my_scope/x
x1: my_scope/x_1
x2: my_scope/x_2
x3: my_scope/x
x4: my_scope_1/x
x5: my_scope/x
True


In [92]:
reset_graph()

text = np.array("Do you want some cafe?".split())

text_tensor = tf.constant(text)

with tf.Session() as sess:
  print(text_tensor.eval())

[b'Do' b'you' b'want' b'some' b'cafe?']
